<a href="https://colab.research.google.com/github/simecek/dspracticum2023/blob/main/lesson09/QLoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -qq git+https://github.com/huggingface/transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
pip install -qq peft bitsandbytes accelerate datasets

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)
from peft import LoraConfig, PeftModel
import datasets

In [ ]:
model = AutoModelForCausalLM.from_pretrained("someone13574/Mistral-7B-v0.1-sharded",
                                             load_in_4bit=True,
                                             low_cpu_mem_usage=True,
                                             device_map="auto")
tokr = AutoTokenizer.from_pretrained("someone13574/Mistral-7B-v0.1-sharded")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
prompt = "Masaryk University is "
toks = tokr(prompt, return_tensors="pt")

In [ ]:
toks

{'input_ids': tensor([[    1,  9183,   628, 28729,  2900,   349, 28705]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
tokr.batch_decode(toks['input_ids'])

['<s> Masaryk University is ']

In [ ]:
%%time
res = model.generate(**toks.to("cuda"), max_new_tokens=25).to('cpu')
res

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


CPU times: user 2.77 s, sys: 253 ms, total: 3.03 s
Wall time: 5.26 s


tensor([[    1,  9183,   628, 28729,  2900,   349, 28705, 28740, 28734, 28734,
          1267,  1571, 28723,   415, 10539,   403, 11573,   297, 28705, 28740,
         28774, 28740, 28774, 28725,   304,   272,   907,  3567,   654, 13048,
           297, 28705]])

In [ ]:
tokr.batch_decode(res)[0]

'<s> Masaryk University is 100 years old. The university was founded in 1919, and the first students were admitted in '

In [ ]:
ds = datasets.load_dataset('knowrohit07/know_sql', revision='f33425d13f9e8aab1b46fa945326e9356d6d5726')

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 78562
    })
})

In [ ]:
trn = ds['train']
trn[3]

{'answer': "SELECT Hosts FROM farm_competition WHERE Theme <> 'Aliens'",
 'question': 'What are the hosts of competitions whose theme is not "Aliens"?',
 'context': 'CREATE TABLE farm_competition (Hosts VARCHAR, Theme VARCHAR)'}

In [ ]:
fmt = """SYSTEM: Use the following contextual information to concisely answer the question.

USER: {}
===
{}
ASSISTANT: """

In [ ]:
def sql_prompt(d): return fmt.format(d["context"], d["question"])
def sql_prompt2(d): return fmt.format(d["context"], d["question"]) + d['answer']

In [ ]:
MICRO_BATCH_SIZE = 4
BATCH_SIZE = 16
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE

EPOCHS = 1
LEARNING_RATE = 3e-4  # the Karpathy constant

CUTOFF_LEN = 1000

LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

In [ ]:
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
tokr.pad_token_id = 2

In [ ]:
def tokenize(prompt):
    result = tokr(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN,
        padding="max_length",

    )
    return {
        "input_ids": result["input_ids"][:-1],
        "attention_mask": result["attention_mask"][:-1],
    }

In [ ]:
data = ds.shuffle().map(lambda x: tokenize(sql_prompt2(x)), num_proc=16)

Map (num_proc=16):   0%|          | 0/78562 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling

trainer = Trainer(
    model=model,
    train_dataset=data['train'],
    args=TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=100,
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,
        logging_steps=20,
        output_dir="./training",
        save_total_limit=1
    ),
    data_collator=DataCollatorForLanguageModeling(tokr, mlm=False),
)
model.config.use_cache = False
trainer.train(resume_from_checkpoint=False)

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
